In [687]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyxirr import xirr
pd.options.display.float_format = '{:,.2f}'.format


# Imputs

In [688]:
df_input=pd.read_csv('./input_fluxo_caixa.csv',header=None,index_col=0)
df_input=df_input.rename(columns={1:'valor_variavel'})

In [689]:
valor_imovel=df_input.loc['valor_imovel','valor_variavel']
entrada_pp=df_input.loc['entrada_pp','valor_variavel']
entrada_por_valor=df_input.loc['entrada_por_valor','valor_variavel']
financiamento_0_ou_1=df_input.loc['financiamento_0_ou_1','valor_variavel']
area=df_input.loc['area','valor_variavel']

# Custos Iniciais

In [690]:
custo_itbi=0.03*valor_imovel
custo_escritura=df_input.loc['custo_escritura','valor_variavel']
custo_registro=df_input.loc['custo_registro','valor_variavel']


# Cálculo financiamento

In [691]:
# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-(n-1)*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+encargos_finaciamento


In [692]:
if financiamento_0_ou_1==1:
    df=pd.DataFrame(columns=['parcela','saldo_devedor'],index=range(0,n_parcelas))
    n_parcelas=int(df_input.loc['n_parcelas','valor_variavel'])                         # 35 anos
    juros_nominal=df_input.loc['juros_nominal','valor_variavel']                        # a.a    
    juros_tr=df_input.loc['tr','valor_variavel']                        # a.a
    juros_nominal=juros_nominal+juros_tr
    custo_avaliacao=df_input.loc['custo_avaliacao','valor_variavel']
    juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

    # he usual way to test for a NaN is to see if it's equal to itself:
    if entrada_por_valor!=entrada_por_valor:
        # Não digitou o valor da entrada
        if entrada_pp!=entrada_pp:
            print('Digite o valor da entrada ou seu percentual.')
        else:
            valor_entrada=valor_imovel*entrada_pp
    else:
        valor_entrada=entrada_por_valor
    
    valor_finaciado=valor_imovel-valor_entrada
    encargos_finaciamento=df_input.loc['encargos_finaciamento','valor_variavel']
    amortizacao=valor_finaciado/n_parcelas
    saldo_devedor=valor_finaciado

else:
    amortizacao=0
    saldo_devedor=0
    n_parcelas=0   
    juros_nominal=0
    custo_avaliacao=0
    juros_efetivo=0  
    valor_finaciado=0
    encargos_finaciamento=0
    valor_entrada=valor_imovel

In [693]:
total_custos_iniciais=valor_entrada+custo_itbi+custo_escritura+custo_registro+custo_avaliacao

In [694]:
for i in range(0,max(24,n_parcelas+1)):             # Se não for financiado seta o df para 24 linhas inicialmente
    if i==0:
        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=0
    else:

        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=parcela(i)

# Custos fixos

In [695]:
condominio=df_input.loc['condominio','valor_variavel']
iptu_am=df_input.loc['iptu_aa','valor_variavel']/12
luz=df_input.loc['luz','valor_variavel']

total_custos_fixos=condominio+iptu_am+luz

In [696]:
for i in range(0,len(df)):
    df.loc[i,'custos_fixos']=total_custos_fixos
    if i==0:
        df.loc[i,'custos_iniciais']=total_custos_iniciais
    else:
        df.loc[i,'custos_iniciais']=0

# Custos com reforma

In [697]:
obra=df_input.loc['obra','valor_variavel']
prazo_obra=df_input.loc['prazo_obra','valor_variavel']                # Prazo da obra em mesês

def reforma(i, n,total):
    if i==0:
        return 0
    elif i<=n:
        return total/n
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'reforma']=reforma(i,prazo_obra,obra)

# Cenários de venda

In [698]:
valor_venda=df_input.loc['valor_venda','valor_variavel']
prazo_venda=int(prazo_obra+df_input.loc['prazo_venda_pos_obra','valor_variavel'])                # Prazo da obra em mesês + venda
comissao=df_input.loc['comissao','valor_variavel']

In [699]:
def venda(i, n,total):
    if i==n:
        print(f'Comissão: {comissao*valor_venda}')
        return -total*(1-comissao)
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'venda']=venda(i,prazo_venda,valor_venda)
df['total']=df.sum(axis=1)
df['total']=df['total']-df['saldo_devedor']

# quitação do saldo devedor e anulação das despesas à partir da venda
for i in range(0,len(df)):
    if i>prazo_venda:
        df.loc[i,'total']=0
    elif i==prazo_venda:
        df.loc[i,'total']=df.loc[i,'total']+df.loc[i,'saldo_devedor']

Comissão: 24000.0


C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_8016\3203338266.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['total']=df.sum(axis=1)


In [700]:
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
start_date = datetime.today().date()+15*delta_d

for i in range(0, len(df)):
    if i==0:
        df.loc[i,'data']=start_date
    else:
        df.loc[i,'data']=start_date+i*delta

df=df[df['total']!=0]
df.to_csv('output.csv', header=True, float_format='%.2f')
df.head(12)

,parcela,saldo_devedor,custos_fixos,custos_iniciais,reforma,venda,total,data
0,0,0,771.67,"664,200.00",0.00,0.00,"664,971.67",2022-08-21
1,0.00,0,771.67,0.00,"16,666.67",0.00,"17,438.33",2022-09-21
2,0.00,0,771.67,0.00,"16,666.67",0.00,"17,438.33",2022-10-21
3,0.00,0,771.67,0.00,"16,666.67",0.00,"17,438.33",2022-11-21
4,0.00,0,771.67,0.00,0.00,0.00,771.67,2022-12-21
5,0.00,0,771.67,0.00,0.00,0.00,771.67,2023-01-21
6,0.00,0,771.67,0.00,0.00,"-776,000.00","-775,228.33",2023-02-21


In [701]:
# calculo da IRR
def cal_irr(x):
    
    dates=list(x['data'])
    amounts=list(x['total'])
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a.(%): ',t_aa*100)
    print('TII a.m.(%): ',t_am*100)
    print('Lucro: ',-x['total'].sum())
    # return (t_aa*100,t_am*100,-x['total'].sum())

In [702]:
cal_irr(df)

TII a.a.(%):  16.620138307198033
TII a.m.(%):  1.2895082487314191
Lucro:  56398.33333333337


In [703]:
valor_m2_aquisicao=valor_imovel/area
print(f'm2 aquisição: {valor_m2_aquisicao}')
valor_m2_venda=valor_venda/area
print(f'm2 reforma: {obra/area}')
print(f'm2 venda: {valor_m2_venda}')


m2 aquisição: 10000.0
m2 reforma: 781.25
m2 venda: 12500.0


In [704]:
11*66+70

796

In [705]:
### projeções de inflação e crescimento econômico para evolução dos preços de venda